# 5. Clustering

This JupyterNotebook is part of an exercise series titled *Clustering*.
The series itself is based on lecture *8. Cluster Analysis*.

There are two parts:

- Part One: Implementing k-means and DBScan
- Part Two: Clustering in the AdventureWorks Database

Recall that we have two exercise groups.
Depending on how each group progresses, some parts of these exercises may not be discussed in its entirety.
If questions arise, ask them in your study group or in our StudOn forum.

## Part One: Implementing k-means and DBScan

<div class="alert alert-block alert-warning">

TODO

</div>

## Part Two: Clustering in the AdventureWorks Database

<div class="alert alert-block alert-warning">

TODO

</div>